# MPEG Audio Layer 3

\begin{itemize}
\item Developed by the MPEG Audio group (Fraunhofer IIS, University of
  Hannover, AT\&T-Bell Labs, Thomson-Brandt and CCETT (Centre Commun
  d'\'Etudes de T\'el\'evision et T\'el\'ecommunications)) in 1992.
\item Used in MP3 players, DVDs, etc.
\item Encoding algorithm:
  \begin{enumerate}
  \item Split the original PCM sequence into 32 PCM sequences, each
    one for a specific frequency subband (subband coding).
  \item Transform each subband using the MDCT.
  \item Determine the quantization values for each subband using a
    psycho-acoustic model.
  \item Lossless compress each MDCT subband sequence using Huffman coding.
\end{enumerate}
\end{itemize}
